In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time
import numpy as np
import tensorflow as tf

In [4]:
!wget -q -O ptb.zip https://ibm.box.com/shared/static/z2yvmhbskc45xd2a9a4kkn6hg4g4kj5r.zip
!unzip -o ptb.zip
!cp ptb/reader.py .
import reader

Archive:  ptb.zip
  inflating: ptb/reader.py           


In [5]:
!wget http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz 
!tar xzf simple-examples.tgz

--2017-11-04 17:38:43--  http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz
Resolving www.fit.vutbr.cz (www.fit.vutbr.cz)... 147.229.9.23, 2001:67c:1220:809::93e5:917
Connecting to www.fit.vutbr.cz (www.fit.vutbr.cz)|147.229.9.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34869662 (33M) [application/x-gtar]
Saving to: ‘simple-examples.tgz’

simple-examples.tgz 100%[===================>]  33.25M   110KB/s    in 8m 14s  

2017-11-04 17:46:58 (68.9 KB/s) - ‘simple-examples.tgz’ saved [34869662/34869662]



In [79]:
init_scale = 0.1
learning_rate = 1.0
max_grad_norm = 5
num_layers = 2
num_steps = 20
hidden_size = 200
max_epoch = 4
max_max_epoch = 13
keep_prob = 1
decay = 0.5
batch_size = 30
vocab_size = 10000
is_training = 1
data_dir = 'simple-examples/data/'

In [7]:
sess = tf.InteractiveSession()

In [9]:
raw_data = reader.ptb_raw_data(data_dir)
train_data, valid_data, test_data, _ = raw_data

In [19]:
itera = reader.ptb_iterator(train_data, batch_size, num_steps)
first_touple = itera.next()
x = first_touple[0]
y = first_touple[1]

In [20]:
x.shape

(30, 20)

In [21]:
x[0:3]

array([[9970, 9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983, 9984,
        9986, 9987, 9988, 9989, 9991, 9992, 9993, 9994, 9995],
       [2654,    6,  334, 2886,    4,    1,  233,  711,  834,   11,  130,
         123,    7,  514,    2,   63,   10,  514,    8,  605],
       [   0, 1071,    4,    0,  185,   24,  368,   20,   31, 3109,  954,
          12,    3,   21,    2, 2915,    2,   12,    3,   21]], dtype=int32)

In [22]:
size = hidden_size

In [23]:
_input_data = tf.placeholder(tf.int32, [batch_size, num_steps])
_targets = tf.placeholder(tf.int32, [batch_size, num_steps])

In [24]:
feed_dict = {_input_data: x, _targets: y}

In [25]:
sess.run(_input_data, feed_dict)

array([[9970, 9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983, 9984,
        9986, 9987, 9988, 9989, 9991, 9992, 9993, 9994, 9995],
       [2654,    6,  334, 2886,    4,    1,  233,  711,  834,   11,  130,
         123,    7,  514,    2,   63,   10,  514,    8,  605],
       [   0, 1071,    4,    0,  185,   24,  368,   20,   31, 3109,  954,
          12,    3,   21,    2, 2915,    2,   12,    3,   21],
       [   3,   71,    4,   27,  246,   60,   11,  215,    4,    1, 1846,
           9,    3,   71,  546,    2, 6505,  162,    6,  104],
       [  93,   25,    6,  261,  681,  251,    0,  278, 3246,   13,  200,
           1,    8,  105, 3360,    1,    4,    0,  536,    4],
       [  20,    6,  954,   12,    3,   21,   78,   14,  977,  726,    0,
          37,   42,   34,    5,  437,  116,  206,  927,    2],
       [  18,  296,    7,  201,   76,    4,  182,  560, 3836,   17,  974,
         975,    6,  942,    4,  156,   77, 1570,  288,  644],
       [  23, 1238,  899,    5,   25,  20

In [26]:
lstm_cell = tf.contrib.rnn.BasicLSTMCell(hidden_size, forget_bias=0.0)
stacked_lstm = tf.contrib.rnn.MultiRNNCell([lstm_cell] * num_layers)

In [27]:
_initial_state = stacked_lstm.zero_state(batch_size, tf.float32)
_initial_state

(LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros:0' shape=(30, 200) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros_1:0' shape=(30, 200) dtype=float32>),
 LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState_1/zeros:0' shape=(30, 200) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState_1/zeros_1:0' shape=(30, 200) dtype=float32>))

In [28]:
sess.run(_initial_state, feed_dict)

(LSTMStateTuple(c=array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32), h=array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)),
 LSTMStateTuple(c=array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32), h=array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0., 

In [29]:
embedding = tf.get_variable("embedding", [vocab_size, hidden_size])

In [30]:
sess.run(tf.global_variables_initializer())
sess.run(embedding, feed_dict)

array([[-0.00906066, -0.00329552, -0.00553765, ..., -0.02034925,
        -0.00844512, -0.02259792],
       [-0.0135624 , -0.02340167,  0.00296221, ...,  0.00993013,
         0.00892542,  0.01307555],
       [ 0.02065075,  0.01868692,  0.01685954, ..., -0.00653633,
         0.00785967,  0.00919427],
       ..., 
       [ 0.0195734 , -0.0189447 ,  0.02025482, ...,  0.00244854,
         0.00244354,  0.0098419 ],
       [-0.00196924,  0.00678424, -0.00997285, ...,  0.02362626,
        -0.00347339,  0.00248653],
       [ 0.00758081,  0.01336546, -0.00324182, ..., -0.02079004,
        -0.02186598,  0.0055558 ]], dtype=float32)

In [32]:
inputs = tf.nn.embedding_lookup(embedding, _input_data)
inputs

<tf.Tensor 'embedding_lookup_1:0' shape=(30, 20, 200) dtype=float32>

In [33]:
sess.run(inputs, feed_dict)

array([[[  1.67964138e-02,   6.04799762e-03,   3.67469154e-03, ...,
          -2.31846496e-02,  -2.35308018e-02,   2.31233612e-02],
        [  1.70437917e-02,  -1.68737285e-02,   1.65801719e-02, ...,
          -6.49345852e-03,  -1.76758058e-02,  -7.16558099e-03],
        [ -2.78907269e-03,   2.24714763e-02,  -8.70266929e-04, ...,
          -5.82855195e-03,  -5.43070957e-03,   9.72737372e-03],
        ..., 
        [ -6.03762455e-03,   1.20601244e-02,   2.34784223e-02, ...,
          -2.60634534e-03,  -7.91697018e-03,   1.48015991e-02],
        [  1.42990276e-02,  -3.15339863e-03,  -2.16326732e-02, ...,
          -3.13481316e-04,  -2.25667264e-02,  -7.28997961e-03],
        [  1.88183822e-02,   9.77063179e-03,   2.39017792e-02, ...,
           1.05959177e-02,  -1.15640629e-02,  -1.07815862e-02]],

       [[ -1.58276502e-02,  -1.05403922e-03,  -1.39244851e-02, ...,
           1.60852969e-02,   1.08305477e-02,   1.87126920e-03],
        [  1.55343451e-02,   1.68615095e-02,  -2.20092870e-0

In [34]:
outputs, new_state = tf.nn.dynamic_rnn(stacked_lstm, inputs, initial_state=_initial_state)

In [35]:
outputs

<tf.Tensor 'rnn/transpose:0' shape=(30, 20, 200) dtype=float32>

In [37]:
sess.run(tf.global_variables_initializer())
sess.run(outputs[0], feed_dict)

array([[  5.28038014e-04,  -1.54598078e-04,  -2.23755138e-04, ...,
          1.31361288e-04,   3.81200080e-04,  -1.93477099e-04],
       [  4.21532139e-04,  -5.36276842e-04,  -3.43302730e-04, ...,
         -1.64015437e-04,   3.11073127e-05,   1.62047785e-04],
       [  2.19064677e-04,  -9.08020302e-04,  -5.33197599e-04, ...,
         -1.90871579e-04,   1.66135971e-04,   1.03238155e-04],
       ..., 
       [  5.16143336e-04,   7.49433239e-04,  -9.26515844e-04, ...,
          2.30391947e-06,  -1.62321463e-04,  -1.47921892e-04],
       [ -3.53590513e-05,   3.48488713e-04,  -2.59241526e-04, ...,
         -3.10192583e-04,  -1.11888701e-04,  -1.86219258e-04],
       [  3.04918678e-04,   1.22855810e-04,   4.30091750e-05, ...,
         -3.18346662e-04,   1.52440436e-04,  -9.93545909e-05]], dtype=float32)

In [38]:
output = tf.reshape(outputs, [-1, size])
output

<tf.Tensor 'Reshape:0' shape=(600, 200) dtype=float32>

In [39]:
sess.run(output[0], feed_dict)

array([  5.28038014e-04,  -1.54598078e-04,  -2.23755138e-04,
        -4.30595101e-05,   2.87399773e-04,   3.98448727e-04,
         3.37762147e-04,   5.00889088e-04,  -6.55235723e-04,
         1.18344484e-04,   4.16582297e-05,   8.98258222e-05,
         1.59812596e-04,   5.28699777e-04,  -4.01756668e-04,
         6.15715035e-05,  -2.43000395e-04,   1.68175015e-04,
         2.97444727e-04,  -1.15601419e-04,   2.00428447e-04,
        -3.18484061e-04,  -1.23442136e-04,   8.82714157e-05,
        -2.53288483e-04,   6.17238175e-06,   9.15494820e-05,
         6.16208592e-04,   1.09454006e-04,  -1.86660836e-05,
         3.02071392e-04,   2.31553728e-04,  -4.11320652e-05,
        -3.40584724e-04,  -9.20751845e-05,   6.42587023e-04,
        -2.33502040e-04,   6.70039677e-04,   2.74759135e-04,
         7.81209455e-05,  -4.17372939e-04,   2.55408697e-04,
        -7.72851490e-05,   3.90585665e-05,  -1.73624197e-04,
         2.63542515e-05,   2.42905026e-05,  -4.08513006e-04,
        -3.78451456e-04,

In [40]:
softmax_w = tf.get_variable("softmax_w", [size, vocab_size])
softmax_b = tf.get_variable("softmax_b", [vocab_size])
logits = tf.matmul(output, softmax_w) + softmax_b

In [41]:
sess.run(tf.global_variables_initializer())
logi = sess.run(logits, feed_dict)
logi.shape

(600, 10000)

In [44]:
First_word_output_probablity = logi[0]
First_word_output_probablity.shape

(10000,)

In [46]:
embedding_array= sess.run(embedding, feed_dict)
np.argmax(First_word_output_probablity)

746

In [47]:
y[0][0]

9971

In [48]:
_targets

<tf.Tensor 'Placeholder_1:0' shape=(30, 20) dtype=int32>

In [49]:
targ = sess.run(tf.reshape(_targets, [-1]), feed_dict)

In [50]:
first_word_target_code= targ[0]
first_word_target_code

9971

In [52]:
first_word_target_vec = sess.run( tf.nn.embedding_lookup(embedding, targ[0]))
first_word_target_vec

array([ -1.16102127e-02,   6.87938184e-04,   1.35919154e-02,
        -9.15668067e-03,  -1.73601173e-02,  -2.27504596e-02,
        -2.36110222e-02,  -9.64871980e-03,  -2.38232315e-05,
        -2.14822870e-02,  -2.99604051e-03,  -1.84585601e-02,
        -3.97298858e-03,  -1.47828683e-02,   1.78000480e-02,
        -1.53268687e-02,  -3.24980542e-03,   1.68571621e-02,
        -1.21433996e-02,  -1.55734057e-02,  -1.91677269e-02,
         2.15151720e-03,   1.75487958e-02,   1.33664720e-02,
         1.51764639e-02,   9.49978828e-04,  -7.40322471e-03,
        -1.52030187e-02,   1.88679211e-02,   2.34593861e-02,
         1.90546215e-02,   8.19662586e-03,   2.04088427e-02,
         2.06931569e-02,  -3.40806693e-03,  -1.37669221e-03,
        -2.19533760e-02,   9.54299793e-03,   4.05992195e-03,
        -2.38196198e-02,   1.12132505e-02,  -2.06005778e-02,
        -9.28800181e-03,  -9.38421115e-03,   2.00445801e-02,
        -1.38140107e-02,   2.33827941e-02,   6.71463273e-03,
         1.33269206e-02,

In [53]:
loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], [tf.reshape(_targets, [-1])],
                                                          [tf.ones([batch_size * num_steps])])

In [54]:
sess.run(loss, feed_dict)

array([ 9.209795  ,  9.19322109,  9.22111416,  9.20602989,  9.20452595,
        9.22083569,  9.1954174 ,  9.19686031,  9.20719433,  9.20785236,
        9.20558262,  9.20939541,  9.20329762,  9.21273327,  9.22187519,
        9.1957798 ,  9.2058115 ,  9.21580887,  9.21448898,  9.20135212,
        9.19718266,  9.22429657,  9.20598316,  9.20951748,  9.21904373,
        9.21196175,  9.21808147,  9.21766853,  9.20844078,  9.19355202,
        9.20308113,  9.21794701,  9.22441101,  9.21420383,  9.19382095,
        9.21950531,  9.22440243,  9.20244026,  9.19492435,  9.22144222,
        9.21341038,  9.20955086,  9.1952858 ,  9.22290039,  9.2253418 ,
        9.22757912,  9.19557858,  9.20654678,  9.21790504,  9.20382881,
        9.19907951,  9.21344376,  9.19389057,  9.21405125,  9.21518993,
        9.21413231,  9.19872665,  9.21329308,  9.19393826,  9.20961189,
        9.20711708,  9.20951176,  9.21288967,  9.19982243,  9.21474075,
        9.20830917,  9.21098518,  9.20942974,  9.2189188 ,  9.21

In [55]:
cost = tf.reduce_sum(loss) / batch_size

sess.run(tf.global_variables_initializer())
sess.run(cost, feed_dict)

184.19798

In [56]:
final_state = new_state

In [57]:
lr = tf.Variable(0.0, trainable= False)
optimizer = tf.train.GradientDescentOptimizer(lr)

In [58]:
tvars = tf.trainable_variables()
tvars

[<tf.Variable 'embedding:0' shape=(10000, 200) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0' shape=(400, 800) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0' shape=(800,) dtype=float32_ref>,
 <tf.Variable 'softmax_w:0' shape=(200, 10000) dtype=float32_ref>,
 <tf.Variable 'softmax_b:0' shape=(10000,) dtype=float32_ref>]

In [59]:
tvars = tvars[3:]

In [60]:
[v.name for v in tvars]

[u'softmax_w:0', u'softmax_b:0']

In [61]:
cost

<tf.Tensor 'truediv:0' shape=() dtype=float32>

In [63]:
tvars

[<tf.Variable 'softmax_w:0' shape=(200, 10000) dtype=float32_ref>,
 <tf.Variable 'softmax_b:0' shape=(10000,) dtype=float32_ref>]

In [64]:
# Toy example (Gradient)
var_x = tf.placeholder(tf.float32)
var_y = tf.placeholder(tf.float32)
func_test = 2.0 * var_x * var_x + 3.0 * var_x * var_y
sess.run(tf.global_variables_initializer())
feed = {var_x: 1.0, var_y: 2.0}
sess.run(func_test, feed)

8.0

In [66]:
var_grad = tf.gradients(func_test, [var_x])
sess.run(var_grad, feed)

[10.0]

In [68]:
var_grad = tf.gradients(func_test, [var_y])
sess.run(var_grad,feed)

[3.0]

In [72]:
grad_t_list = tf.gradients(cost, tvars)

In [73]:
max_grad_norm

5

In [75]:
grads, _ = tf.clip_by_global_norm(grad_t_list, max_grad_norm)
grads

[<tf.Tensor 'clip_by_global_norm_1/clip_by_global_norm_1/_0:0' shape=(200, 10000) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm_1/clip_by_global_norm_1/_1:0' shape=(10000,) dtype=float32>]

In [76]:
sess.run(grads, feed_dict)

[array([[  2.94609548e-04,   3.76086682e-04,   1.96285153e-04, ...,
          -3.59663971e-07,  -3.58780341e-07,  -3.51121685e-07],
        [  1.30369066e-04,   4.24927348e-05,  -1.54847265e-04, ...,
          -1.24238426e-07,  -1.23917914e-07,  -1.21292800e-07],
        [  2.56144540e-05,   6.01838401e-05,   1.24944621e-04, ...,
           1.93276453e-08,   1.92546583e-08,   1.88615648e-08],
        ..., 
        [ -2.55317922e-04,  -2.42163165e-04,  -1.89787257e-04, ...,
           3.97985531e-07,   3.97018511e-07,   3.88513627e-07],
        [  1.62385659e-05,   1.15497482e-04,   8.28100019e-05, ...,
          -8.96525094e-08,  -8.94086298e-08,  -8.75088446e-08],
        [ -2.60127854e-04,  -1.99975562e-04,  -2.00370967e-04, ...,
           4.68515196e-07,   4.67331489e-07,   4.57362233e-07]], dtype=float32),
 array([-0.797997  , -1.03135109, -1.03133023, ...,  0.00201682,
         0.00201183,  0.00196875], dtype=float32)]

In [77]:
train_op = optimizer.apply_gradients(zip(grads, tvars))

In [78]:
sess.run(tf.global_variables_initializer())
sess.run(train_op, feed_dict)

In [103]:
class PTBModel(object):

    def __init__(self, is_training):

        self.batch_size = batch_size
        self.num_steps = num_steps
        size = hidden_size
        self.vocab_size = vocab_size

        self._input_data = tf.placeholder(tf.int32, [batch_size, num_steps])
        self._targets = tf.placeholder(tf.int32, [batch_size, num_steps])

        lstm_cell = tf.contrib.rnn.BasicLSTMCell(size, forget_bias=0.0)

        if is_training and keep_prob < 1:
            lstm_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=keep_prob)
        
        stacked_lstm = tf.contrib.rnn.MultiRNNCell([lstm_cell] * num_layers)

        self._initial_state = stacked_lstm.zero_state(batch_size, tf.float32)

        with tf.device("/gpu:0"):
            embedding = tf.get_variable("embedding", [vocab_size, size])
            inputs = tf.nn.embedding_lookup(embedding, self._input_data)

        if is_training and keep_prob < 1:
            inputs = tf.nn.dropout(inputs, keep_prob)

        outputs, state = tf.nn.dynamic_rnn(stacked_lstm, inputs, initial_state=self._initial_state)

        output = tf.reshape(outputs, [-1, size])
        softmax_w = tf.get_variable("softmax_w", [size, vocab_size])
        softmax_b = tf.get_variable("softmax_b", [vocab_size])
        logits = tf.matmul(output, softmax_w) + softmax_b

        loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], [tf.reshape(self._targets, [-1])],
                                                      [tf.ones([batch_size * num_steps])])
        self._cost = cost = tf.reduce_sum(loss) / batch_size

        self._final_state = state

        if not is_training:
            return

        self._lr = tf.Variable(0.0, trainable=False)
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), max_grad_norm)
        optimizer = tf.train.GradientDescentOptimizer(self.lr)
        self._train_op = optimizer.apply_gradients(zip(grads, tvars))

    # Helper functions for our LSTM RNN class

    def assign_lr(self, session, lr_value):
        session.run(tf.assign(self.lr, lr_value))

    @property
    def input_data(self):
        return self._input_data

    @property
    def targets(self):
        return self._targets

    @property
    def initial_state(self):
        return self._initial_state

    @property
    def cost(self):
        return self._cost

    @property
    def final_state(self):
        return self._final_state

    @property
    def lr(self):
        return self._lr

    @property
    def train_op(self):
        return self._train_op

In [104]:
def run_epoch(session, m, data, eval_op, verbose=False):

    epoch_size = ((len(data) // m.batch_size) - 1) // m.num_steps
    start_time = time.time()
    costs = 0.0
    iters = 0

    state = session.run(m.initial_state)
    
    for step, (x, y) in enumerate(reader.ptb_iterator(data, m.batch_size, m.num_steps)):
 
        cost, state, _ = session.run([m.cost, m.final_state, eval_op],
                                     {m.input_data: x,
                                      m.targets: y,
                                      m.initial_state: state})
        
        costs += cost
        
        iters += m.num_steps

        if verbose and step % (epoch_size // 10) == 10:
            print("%.3f perplexity: %.3f speed: %.0f wps" % (step * 1.0 / epoch_size, np.exp(costs / iters),
              iters * m.batch_size / (time.time() - start_time)))

    return np.exp(costs / iters)

In [105]:
raw_data = reader.ptb_raw_data(data_dir)
train_data, valid_data, test_data, _ = raw_data

In [106]:
with tf.Graph().as_default(), tf.Session() as session:
    initializer = tf.random_uniform_initializer(-init_scale,init_scale)
    
    with tf.variable_scope("model", reuse=None, initializer=initializer):
        m = PTBModel(is_training=True)
        
    with tf.variable_scope("model", reuse=True, initializer=initializer):
        mvalid = PTBModel(is_training=False)
        mtest = PTBModel(is_training=False)

    tf.global_variables_initializer().run()

    for i in range(max_max_epoch):
        lr_decay = decay ** max(i - max_epoch, 0.0)
        
        m.assign_lr(session, learning_rate * lr_decay)

        print("Epoch %d : Learning rate: %.3f" % (i + 1, session.run(m.lr)))
        
        train_perplexity = run_epoch(session, m, train_data, m.train_op,
                                   verbose=True)
        print("Epoch %d : Train Perplexity: %.3f" % (i + 1, train_perplexity))
        
        valid_perplexity = run_epoch(session, mvalid, valid_data, tf.no_op())
        print("Epoch %d : Valid Perplexity: %.3f" % (i + 1, valid_perplexity))
    
    test_perplexity = run_epoch(session, mtest, test_data, tf.no_op())
    
    print("Test Perplexity: %.3f" % test_perplexity)

Epoch 1 : Learning rate: 1.000
0.006 perplexity: 5884.353 speed: 30755 wps
0.106 perplexity: 1068.353 speed: 34392 wps
0.205 perplexity: 814.324 speed: 35183 wps
0.305 perplexity: 664.918 speed: 35226 wps
0.404 perplexity: 560.777 speed: 35319 wps
0.504 perplexity: 495.421 speed: 35475 wps
0.603 perplexity: 445.998 speed: 35379 wps
0.702 perplexity: 409.485 speed: 34984 wps
0.802 perplexity: 381.497 speed: 34994 wps
0.901 perplexity: 355.566 speed: 35054 wps
Epoch 1 : Train Perplexity: 335.193
Epoch 1 : Valid Perplexity: 200.443
Epoch 2 : Learning rate: 1.000
0.006 perplexity: 223.254 speed: 30751 wps
0.106 perplexity: 198.062 speed: 35125 wps
0.205 perplexity: 190.227 speed: 34613 wps
0.305 perplexity: 182.438 speed: 34566 wps
0.404 perplexity: 174.148 speed: 34506 wps
0.504 perplexity: 170.804 speed: 34518 wps
0.603 perplexity: 167.147 speed: 34514 wps
0.702 perplexity: 164.015 speed: 34619 wps
0.802 perplexity: 161.778 speed: 34810 wps
0.901 perplexity: 157.913 speed: 34954 wps
Epoc